# MHW Notebook: TimeSeries
**Version**: v1.0-beta 

**Last update**: 2025-01-31

**Authors**: ...


This notebook is designed for the operational analysis of Marine Heatwave (MHW) detection in the Mediterranean Sea by comparing climatological baselines to Sea Surface Temperature (SST) data from reprocessed (REP) or near-real-time (NRT) satellite observations, as well as model forecast data (MFS). It connects to the Copernicus Marine Service (CMEMS) via the copernicusmarine API.

Data is area-averaged using grid cell areas and visualized as a time series plot to enhance the detection and analysis of MHW events. Efficient processing of large spatial datasets is enabled by xarray and numpy, while the wavesnspikes algorithm supports real-time heatwave detection.

In [ ]:
import os, time, warnings
from datetime import date, datetime, timedelta

import ipywidgets as widgets
from IPython.display import display

import numpy as np
import xarray as xr
from wavesnspikes import wns
import copernicusmarine

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import cartopy.crs as ccrs
import cartopy.feature as cfeature

warnings.filterwarnings("ignore") # Ignore all warnings

### 1. Select the CMEMS dateset of interest
- Mediterranean Satellite Reprocessed (REP) Sea Surface Temperature (SST) with data from 1982
- Mediterranean Satellite Near Real Time SST (NRT) with data from 2008
- ~~Model Mediterranean Forecasting System (MFS) that provides daily forecasts of oceanic variables from most recent years and 10 days forecast~~ (not yet available)

- Climatology data: Long-term averages (from 1987–2021) to provide a baseline for comparing the CMEMS data.

In [ ]:
# CMEMS products metadata
CMEMS_datasets_options = {'cmems_SST_MED_SST_L4_REP_OBSERVATIONS_010_021':{'varname':  "analysed_sst", "prod_type": 'REP',
                                                                         'grid_file':"prev/cell_areas_CMS.nc",
                                                                         'clim_file':"CMS_SST_Climatology_19872021.nc",
                                                                         'region_folder':'prev'},
                        'SST_MED_SST_L4_NRT_OBSERVATIONS_010_004_a_V2': {'varname':  "analysed_sst", "prod_type": 'NRT',
                                                                         'grid_file':"2024/cell_areas_CMS_NRT.nc",
                                                                         'clim_file':"CMS_SST_Climatology_19872021_rect00625.nc",
                                                                         'region_folder':'2024'},
                        # 'cmems_mod_med_phy-tem_anfc_4.2km_P1D-m':       {'varname':  "thetao", "prod_type": 'MFS',
                        #                                                  'grid_file':"MFS_CMS/cell_areas_MFS_CMS.nc",
                        #                                                  'clim_file':"MEDREA_Climatology_19872021.nc",
                        #                                                  'region_folder':'MFS_CMS'}
                        }


# CMEMS datasets selection
print('Please select a Copernicus Marine Service (CMEMS) dataset:')
datasets_dropdown = widgets.Dropdown(options = CMEMS_datasets_options.keys(), description = 'Dataset:', disabled = False)
display(datasets_dropdown) # Display the dropdown

### Paths Settings 

In [ ]:
# areas and grids
area_path = os.path.expanduser("~/blue-cloud-dataspace/MEI/CMCC/MHW/input/area/")
# area_path = "/workspace/VREFolders/MarineEnvironmentalIndicators/input_datasets/mhw/area"
# climatologies
clim_path = os.path.expanduser("~/blue-cloud-dataspace/MEI/CMCC/MHW/input/clim/")
# clim_path = "/workspace/VREFolders/MarineEnvironmentalIndicators/input_datasets/mhw/clim"
# regions
reg_path  = os.path.expanduser("~/blue-cloud-dataspace/MEI/CMCC/MHW/input/region/")
region_files = os.listdir(os.path.join(reg_path,"MFS_CMS"))

out_dir   = "/workspace/MHW_figures/TimeSeries/"
os.makedirs(out_dir, exist_ok=True) # Check if the directory exists; create it if it doesn't

In [ ]:
## MHW settings
time_dim  = 'time'
ndays_min = 30

### 2. Load and set
- Query copernicusmarine api for selected dataset
- Open climatology dataset
- Select date of interest
- Select region of interest

In [ ]:
dataset_id = datasets_dropdown.value
varname    = CMEMS_datasets_options[dataset_id]['varname']
prod       = CMEMS_datasets_options[dataset_id]['prod_type']
print('Selected dataset -> %s'%(dataset_id))

# Querying copernicusmarine api
print("Querying copernicusmarine api...")
t0=time.time()
params = {"credentials_file": "bc2026_copernicusmarine-credentials",
          "dataset_id": dataset_id, "variables" : [varname], "maximum_depth": 1.5,}
if prod == 'MFS': params["minimum_longitude"] = -6 # this limit is set because of the grid of the available region masks 
cms_rawdataset = copernicusmarine.open_dataset(**params)
print('\tElapsed time: %ss'%(round(time.time()-t0,1)))

# Setting the date range
date_min = datetime.utcfromtimestamp(min(cms_rawdataset[time_dim].values).astype('datetime64[s]').astype(int)).date()
date_max = datetime.utcfromtimestamp(max(cms_rawdataset[time_dim].values).astype('datetime64[s]').astype(int)).date()
start_datepicker = widgets.DatePicker(description='Start', disabled=False, value=date_max-timedelta(ndays_min), min=date_min, max=date_max)
end_datepicker   = widgets.DatePicker(description='End', disabled=False, value=date_max, min=date_min, max=date_max)
print('\nPlease select the date range:\nLimits of the dataset -> from %s to %s' %(date_min, date_max))
display(start_datepicker)
display(end_datepicker)

# Setting the boundary method
print('Please select a boundary method (box or regions):')
boundary_dropdown = widgets.Dropdown(options=['Rectangle Box','Pre-defined regions'], description='Method:', disabled=False)
display(boundary_dropdown) # Display the dropdown

In [ ]:
# Climatology and grid area file
print("\nOpening cell area...")
t0=time.time()
area_rawdataset = area_clim_rawdataset = xr.open_dataset(os.path.join(area_path,CMEMS_datasets_options[dataset_id]['grid_file']))
print(f"\tDone ({time.time() - t0:.1f}s).")
print("\nOpening climatology files...")
t0=time.time()
clim_rawdataset = xr.open_dataset(os.path.join(clim_path,CMEMS_datasets_options[dataset_id]['clim_file']))
if prod == 'MFS': area_clim_rawdataset = xr.open_dataset(os.path.join(area_path,"MFS/cell_areas_MEDREA.nc")) # climatology and MFS on different grids
print(f"\tDone ({time.time() - t0:.1f}s).")

In [ ]:
# Dates check
date_start = start_datepicker.value
date_end   = end_datepicker.value 
if (date_end-date_start).days < ndays_min:
    print('Minimum range of %s days not respected OR Start date is after End date.\nSetting Start date to %s days before End date.'%(ndays_min,ndays_min))
    date_start = date_end - timedelta(ndays_min)
print('\nDate range: %s to %s\n'%(date_start,date_end))

# Boundary check
print('Boundary method selected -> %s'%boundary_dropdown.value)
filter_by_box = False
if boundary_dropdown.value == 'Rectangle Box':
    filter_by_box = True
    print('\nInsert the coordinates of the region of interest:\nThe standard values are the dataset limits.')
    n_dec = 3
    lon_min, lon_max = np.round(cms_rawdataset.longitude.min().item(),n_dec), np.round(cms_rawdataset.longitude.max().item(),n_dec)
    lat_min, lat_max = np.round(cms_rawdataset.latitude.min().item(),n_dec),  np.round(cms_rawdataset.latitude.max().item(),n_dec)
    lonmin_input = widgets.BoundedFloatText(description='Minimum:',value=lon_min,min=lon_min,max=lon_max,)
    lonmax_input = widgets.BoundedFloatText(description='Maximum:',value=lon_max,min=lon_min,max=lon_max,)
    latmin_input = widgets.BoundedFloatText(description='Minimum:',value=lat_min,min=lat_min,max=lat_max,)
    latmax_input = widgets.BoundedFloatText(description='Maximum:',value=lat_max,min=lat_min,max=lat_max,)
    # Display the widgets to select the region of interest
    print('Longitude:')
    display(widgets.VBox([lonmin_input, lonmax_input]))
    print('Latitude:')
    display(widgets.VBox([latmin_input, latmax_input]))
elif boundary_dropdown.value == 'Pre-defined regions':
    regions = sorted([os.path.basename(file).split('_region.nc')[0] for file in region_files])
    print('Select the Pre-defined region:')
    region_dropdown = widgets.Dropdown(options=regions, description='Region:', disabled=False)
    display(region_dropdown)

In [ ]:
# Plot selected region [not mandatory, just for visualization]
projection = ccrs.PlateCarree()
fig, axes  = plt.subplots(nrows=1,ncols=1,figsize=(10,6),subplot_kw={'projection': projection} )
fig.subplots_adjust(bottom=0.02, top=0.92, left=0.02, right=0.87, wspace=0.05, hspace=0.05)
axes._autoscaleXon = axes._autoscaleYon = False
axes.coastlines()
axes.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', edgecolor='black', facecolor='lightgrey'))
axes.set_extent([lon_min, lon_max, lat_min, lat_max], crs=projection) # [west lon, east lon, south lat, north]
gl = axes.gridlines(draw_labels=True, crs=ccrs.PlateCarree(), color='gray', linestyle='--', linewidth=0.2)
gl.xlabel_style = gl.ylabel_style = {'size': 10, 'color': 'black'}
gl.top_labels = gl.right_labels = False  # Disable top and right labels
if filter_by_box:     # Add the box to the map
    from matplotlib.patches import Rectangle
    axes.set_title('Selected Box:\nLons -> %s to %s\nLats -> %s to %s'%(lonmin_input.value,lonmax_input.value,latmin_input.value,latmax_input.value))
    axes.add_patch(Rectangle((lonmin_input.value, latmin_input.value),  # Lower-left corner
                              lonmax_input.value - lonmin_input.value,  # Width
                              latmax_input.value - latmin_input.value,  # Height
                              linewidth=2, edgecolor='red', facecolor='none', transform=projection,zorder=10))
else:                 # Open and add the region mask
    axes.set_title('Selected region -> %s'%(region_dropdown.value))
    reg_mask = xr.open_dataset(os.path.join(reg_path,"MFS_CMS",region_dropdown.value+'_region.nc'))
    valid_points = reg_mask.where(reg_mask.index_region == 1, drop=True).stack(points=("lat", "lon")).dropna("points")
    axes.scatter(valid_points.lon.values, valid_points.lat.values, color="steelblue", transform=projection, label="Region Mask")
plt.show()

### 3. Filtering and Processing the Datasets
- Date and region of interest filters
- Compute Anomaly and MHW mask

In [ ]:
def extract_clim_date_range(ds, dates_list, time_dim="days"):
    """
    Extract a specific range of dates from a climatology dataset.
    Parameters:
    - ds (xr.Dataset or xr.DataArray): The input dataset or data array with time_dim dimensions or coordinates.
    - dates_list (list of datetime): The list of the dates to extract
    - time_dim (str): The name of the time dimension in the dataset.
    Returns: The extracted subset of the dataset or data array as xr.Dataset or xr.DataArray.
    """
    # Helper functions to calculate climatological day-of-year with adjust for leap years
    def is_leap_year(year): return (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0)
    
    def to_climatological_day(date):
        start_of_year = datetime(date.year, 1, 1)
        current_date  = datetime(date.year, date.month, date.day)
        dayofyear = (current_date - start_of_year).days
        if date.month == 2 and date.day == 29:           dayofyear = 58 # If 29 Feb return 28 Feb dayofyear value
        elif date.month > 2 and is_leap_year(date.year): dayofyear -= 1 # If leap year and date is after Feb 28, subtract one day
        return dayofyear
    
    extracted_data = [ds.where(ds[time_dim] == to_climatological_day(date), drop=True) for date in dates_list]
    result = xr.concat(extracted_data, dim=time_dim)
    result = result.assign_coords({time_dim: dates_list})
    return result

def filter_box(ds, lon_min, lon_max, lat_min, lat_max, lon_var='lon',lat_var='lat', output=''):
    """
    Filters an xarray dataset based on a bounding box of latitude and longitude.
    Parameters:
        ds (xarray.Dataset): The input dataset with 2D lat and lon coordinates.
        lon_min,... (float): coordinates limits the bounding box.
    Returns: The filtered xarray.Dataset.
    """
    # Create a mask for the bounding box
    mask = ((ds[lon_var] >= lon_min) & (ds[lon_var] <= lon_max) & (ds[lat_var] >= lat_min) & (ds[lat_var] <= lat_max))
    if output == 'mask': return  mask.astype(int)#.broadcast_like(ds)
    else: return ds.where(mask, drop=True)  # Apply the mask to the dataset and drop values outside the range

def standardize_dim_names(ds, lat_name='latitude', lon_name='longitude', time_name='time'):
    """Standardize the dimension names of a dataset.
    Parameters:
    - ds (xarray.Dataset or xarray.DataArray): Input dataset or data array.
    - *_name (str): standardized dimension name.
    Returns: - xarray.Dataset or xarray.DataArray: Dataset with standardized dimension names.
    """
    # Define a mapping of old dimension names to new ones
    rename_map = {}
    if 'latitude'    in ds.dims: rename_map['latitude']  = lat_name
    elif 'Latitude'  in ds.dims: rename_map['Latitude']  = lat_name
    elif 'lat'       in ds.dims: rename_map['lat']       = lat_name
    if 'longitude'   in ds.dims: rename_map['longitude'] = lon_name
    elif 'Longitude' in ds.dims: rename_map['Longitude'] = lon_name
    elif 'lon'       in ds.dims: rename_map['lon']       = lon_name
    if 'time'        in ds.dims: rename_map['time']      = time_name
    elif 'days'      in ds.dims: rename_map['days']      = time_name
    elif 'dayofyear' in ds.dims: rename_map['dayofyear'] = time_name
    ds = ds.rename(rename_map) # Rename dimensions
    if time_name in ds.dims: ds = ds.transpose(time_name, lat_name, lon_name)
    else: ds = ds.transpose(lat_name, lon_name)
    return ds

def extract_TS(data,region,area):
    region_3D = np.tile(region,(data.shape[0],1,1))
    area_3D   = np.tile(area,(data.shape[0],1,1))
    area_mask_3D = np.ma.masked_where(region_3D==0,area_3D)
    data_mask    = np.ma.masked_where(region_3D==0,data)
    data_mask    = np.ma.masked_where(np.isnan(data),data_mask)
    data_mask    = np.ma.masked_where(data==0,data_mask)
    area_mask_3D.mask = data_mask.mask # forcing nan mask of data_mask into area_mask_3D (necessary for box masks)
    return np.sum(data_mask*area_mask_3D,axis=(1,2))/np.sum(area_mask_3D,axis=(1,2)) # calculates area-average

In [ ]:
# Filtering and processing
dataset_cms  = cms_rawdataset.copy()
dataset_clim = clim_rawdataset.copy()
cell_area      = area_rawdataset.cell_area
cell_area_clim = area_clim_rawdataset.cell_area
if 'depth' in dataset_cms.dims and dataset_cms.sizes['depth'] == 1: dataset_cms = dataset_cms.squeeze(dim='depth')

print('Filtering...')
print('\tDates filter -> %s to %s (%s days)'%(date_start, date_end, (date_end-date_start).days))
dates_list = [date_start + timedelta(days=i) for i in range((date_end - date_start).days + 1)]
dataset_cms  = dataset_cms.sel({time_dim:slice(dates_list[0], dates_list[-1])}) # Date filter
dataset_clim = extract_clim_date_range(dataset_clim, dates_list)
#clim_ds = clim_ds.where(clim_ds != 9.96920997e+36, np.nan)

if filter_by_box: 
    print('\tBox filter   -> Lons [%s to %s] Lats [%s to %s]'%(lonmin_input.value,lonmax_input.value,latmin_input.value,latmax_input.value))
    region_mask = region_mask_clim = filter_box(dataset_cms,lonmin_input.value,lonmax_input.value,latmin_input.value,latmax_input.value,lat_var='latitude',lon_var='longitude', output='mask')
else:
    region_name = region_dropdown.value
    print('\tRegion -> %s'%region_name)
    region_mask   =  region_mask_clim = xr.open_dataset(os.path.join(reg_path,CMEMS_datasets_options[dataset_id]['region_folder'],region_name+'_region.nc')).index_region
    if prod == 'MFS':region_mask_clim = xr.open_dataset(os.path.join(reg_path,'MFS',region_name+'_region.nc')).index_region # climatology and MFS on different grids

# Standardize coordinates (lat,lon) dimensions names 
dataset_cms = standardize_dim_names(dataset_cms)
region_mask = standardize_dim_names(region_mask)
cell_area   = standardize_dim_names(cell_area)
dataset_clim     = standardize_dim_names(dataset_clim)
region_mask_clim = standardize_dim_names(region_mask_clim)
cell_area_clim   = standardize_dim_names(cell_area_clim)  
# extract_TS for each variable of the datasets
print('Applying masks and computing area-average...')
t0=time.time()
averaged_vars = {}
print('\tCMEMS data...')
for var_name, var_data in dataset_cms.data_vars.items():
    if var_data.min() > 100: var_data = var_data - 273.15  # Kelvin units check
    averaged_vars[var_name] = xr.DataArray(extract_TS(var_data, region_mask, cell_area), 
                                            dims=[time_dim], coords={time_dim: var_data[time_dim]},name=var_name)
print('\tCLIM data...')
for var_name, var_data in dataset_clim.data_vars.items():
    if var_data.min() > 100: var_data = var_data - 273.15  # Kelvin units check
    averaged_vars[var_name] = xr.DataArray(extract_TS(var_data, region_mask_clim, cell_area_clim), 
                                            dims=[time_dim], coords={time_dim: var_data[time_dim]},name=var_name)
print('\tElapsed time: %ss'%(round(time.time()-t0,1)))
processed_dataset = xr.Dataset(averaged_vars) # Combine all variables into a single dataset
print('Detecting Marine HeatSpikes (MHS) and Marine HeatWaves (MHW)...')
t0=time.time()
MHS, MHW = wns(processed_dataset[varname].values, processed_dataset.pc90.values)
processed_dataset['MHS'] = xr.DataArray(MHS, dims=processed_dataset[varname].dims,  coords=processed_dataset[varname].coords)
processed_dataset['MHW'] = xr.DataArray(MHW, dims=processed_dataset[varname].dims,  coords=processed_dataset[varname].coords)
print('\tElapsed time: %ss'%(round(time.time()-t0,1)))
daterange_str = '%s_to_%s'%(str(date_start).replace('-', ''), str(date_end).replace('-', ''))
if filter_by_box: region_str = 'box[Lon%sto%s][Lat%sto%s]'%(lonmin_input.value,lonmax_input.value,latmin_input.value,latmax_input.value)
else: region_str = region_name
# out_filename = "MHWtimeseries_CMEMS_%s_%s_%s.nc"%(prod,daterange_str,region_str)
out_filename = "test.nc"
output_file = os.path.join(out_dir,out_filename)
processed_dataset.to_netcdf(output_file)
print(f"\tNetCDF file saved to {output_file}")

### 4. TimeSeries plot
- SST climatology, 90th percentile and daily area-averaged data from CMEMS product
- Filled area for detected MHS (yellow) and MHW (orange)
- matplotlib

In [ ]:
# PLOTTING
figname = 'MHW_timeseries_%s_%s_%s.png'%(prod,daterange_str,region_str)

#figsize = (8,4.5)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))
fig.subplots_adjust(bottom=0.1, top=0.92, left=0.1, right=0.98, wspace=0.1, hspace=0.1)
ax.set_title("Marine Heat Waves\n%s\n%s"%(dataset_id,region_str),fontweight='bold')
xrange=(date_start,date_end)

ax.grid(alpha=0.6)
ax.plot(processed_dataset[time_dim],processed_dataset[varname],'-',color='darkgrey',lw=3,label="CMEMS data")
ax.plot(processed_dataset[time_dim],processed_dataset.pc90,'--',color='darkred',label="MHW Threshold")
ax.plot(processed_dataset[time_dim],processed_dataset.clim,'-',color='darkred',label="Average 1987-2021")
    
ax.fill_between(processed_dataset[time_dim],processed_dataset.pc90,processed_dataset[varname],where=processed_dataset['MHS'],color='yellow')
ax.fill_between(processed_dataset[time_dim],processed_dataset.pc90,processed_dataset[varname],where=processed_dataset['MHW'],color='orange')

ax.set_ylabel("Temperature ($^{o}C)$")
ax.set_xlim(xrange)
ax.xaxis.set_major_locator(MaxNLocator(nbins=8))

ax.legend(loc="lower center",bbox_to_anchor=(.5, -.2), ncol=3)
fig.tight_layout()
fig.savefig(output_file.replace('.nc','.png'),dpi=300)
print(f"\tPNG figure saved at '{output_file.replace('.nc','.png')}'\n")